In [ ]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.6 MB/s eta 0:00:00


In [ ]:
#test GPU
import torch
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'No GPU detected')

True
Tesla T4


In [ ]:
!pip install scikit-learn

In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import transformers
from transformers import AutoModel, BertTokenizerFast

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
df = pd.read_csv("Artificial Intelligence and Machine Learning (AIML)-Enabled Medical Devices  FDA (1).csv")
df.head()

,Artificial Intelligence and Machine Learning (AI/ML)-Enabled Medical Devices | FDA,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5
0,Date of Final Decision,Submission Number,Device,Company,Panel (Lead),Primary Product Code
1,07/29/2022,K213760,ABMD Software,HeartLung Corporation,Radiology,KGI
2,07/29/2022,K220961,Deep Learning Image Reconstruction,GE Healthcare Japan Corporation,Radiology,JAK
3,07/28/2022,K213998,cvi42 Auto Imaging Software Application,Circle Cardiovascular Imaging Inc,Radiology,QIH
4,07/28/2022,K221923,Swoop Portable MR Imaging System,"Hyperfine, Inc.",Radiology,LNH


In [ ]:
#upload dataset
from google.colab import files
uploaded = files.upload()
list(uploaded.keys())

Saving system_design_qa.json to system_design_qa.json


['system_design_qa.json']

In [ ]:
from datasets import load_dataset
dataset = load_dataset("json", data_files={"train": list(uploaded.keys())[0]})

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
tokenizer = AutoTokenizer.from_pretrained("deepset/roberta-base-squad2")
model = AutoModelForQuestionAnswering.from_pretrained("deepset/roberta-base-squad2")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/79.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/496M [00:00<?, ?B/s]

In [ ]:
import json
from datasets import Dataset, DatasetDict
from transformers import AutoTokenizer
from typing import List, Dict, Any

In [ ]:
#Load + flatten utility (handles nested SQuAD automatically)
def load_and_flatten(files: List[str]) -> Dataset:
  rows = []
  for fp in files:
    with open(fp, "r") as f:
      data = json.load(f)

    if isinstance(data, dict) and "data" in data:
      for item in data["data"]:
        title = item.get("title", "")
        for para in item.get("paragraphs", []):
          context = para.get("context", "")
          for qa in para.get("qas", []):
            q = qa.get("question", "")
            qa_id = qa.get("id", "")
            ans = qa.get("answers", [])

            if isinstance(ans, list):
              texts = [a["text"] for a in ans]
              starts = [a["answer_start"] for a in ans]
              answers = {"text": texts, "answer_start": starts}
            else:
              answers = {
                  "text": ans.get("text", []),
                  "answer_start": ans.get("answer_start", [])
              }
            rows.append({
                "id": qa_id,
                "title": title,
                "context": context,
                "question": q,
                "answers": answers
            })
    elif isinstance(data, list):
      for r in data:
        answers = r.get("answers", {})
        if isinstance(answers, list):
          answers = {
              "text": [a["text"] for a in answers],
              "answer_start": [a["answer_start"] for a in answers]
          }
        rows.append({
            "id": r.get("id", ""),
            "title": r.get("title", ""),
            "context": r.get("context"),
            "question": r.get("question"),
            "answers": answers
        })
    else:
      raise ValueError(f"Unsupported data format in {fp}")
  return Dataset.from_list(rows)
files = list(uploaded.keys())
dataset = load_and_flatten(files)
print(dataset)
print(dataset[0])

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 6
})
{'id': 'rivanna_q1', 'title': 'Rivanna_Accuro', 'context': 'Hardware: Accuro, Consumables, True-View Trolley, Accuro XV, ultrasound guidance platform, Accuro® 3S diagnostic ultrasound system, Trolley, cords, adapters, Locator kits, Locator needle guides, imaging-based medical technology. Software Components: automated landmark identification, AI-enabled image guidance, CADe/x AI module, AI-driven imaging software, True-View, Upgrade, proprietary technology solutions. AI Models: AI-enabled image guidance, CADe/x AI module, AI-driven imaging software. Data Pipelines: No cloud subscription required, crucial data, midline and intervertebral space detection, imaging-based solution, aggregate usage data, collect information from third-party sources. User Interface: Pinpoint key neuraxial targets, integrated clinician support, True-View, Website(s), online contact form, social network accounts. Integ

In [ ]:
#Train/ Validation split (90/10 by default)
ds = dataset.train_test_split(test_size=0.1, seed=42)
raw = DatasetDict({"train": ds["train"], "validation": ds["test"]})
raw

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 5
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 1
    })
})

In [ ]:
#Tokenize in SQuaD-style
from transformers import AutoTokenizer

model_ckpt = "deepset/roberta-base-squad2"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt, use_fast=True)

def preprocess_batch(batch):
    questions = [q.lstrip() for q in batch["question"]]
    contexts = batch["context"]
    answers = batch["answers"]

    tokenized_inputs = tokenizer(
        questions,
        contexts,
        truncation="only_second", # Truncate only the context
        padding="max_length",
        max_length=512,
        return_offsets_mapping=True, # Needed to map token positions to character positions
    )

    input_ids = tokenized_inputs["input_ids"]
    attention_mask = tokenized_inputs["attention_mask"]
    offset_mapping = tokenized_inputs["offset_mapping"]

    start_positions = []
    end_positions = []

    for i, offsets in enumerate(offset_mapping):
        input_id = input_ids[i]
        cls_index = input_id.index(tokenizer.cls_token_id)

        # Find the start and end of the context
        sequence_ids = tokenized_inputs.sequence_ids(i)
        context_start = sequence_ids.index(1)
        context_end = len(sequence_ids) - 1 - list(reversed(sequence_ids)).index(1)

        # If there's no answer, label with the CLS index
        if not answers[i]["answer_start"]:
            start_positions.append(cls_index)
            end_positions.append(cls_index)
        else:
            # Start/end character index of the answer in the text
            start_char = answers[i]["answer_start"][0]
            end_char = start_char + len(answers[i]["text"][0])

            # Start token index of the answer in the context
            token_start_index = context_start
            while token_start_index < len(input_id) and offsets[token_start_index][0] <= start_char:
                token_start_index += 1
            token_start_index -= 1

            # End token index of the answer in the context
            token_end_index = context_end
            while token_end_index >= 0 and offsets[token_end_index][1] >= end_char:
                token_end_index -= 1
            token_end_index += 1

            # If the tokenized answer does not match the character answer span,
            # or if the answer is outside the context, label with the CLS index
            if not (token_start_index >= context_start and
                    token_end_index <= context_end and
                    offsets[token_start_index][0] == start_char and
                    offsets[token_end_index][1] == end_char):
                start_positions.append(cls_index)
                end_positions.append(cls_index)
            else:
                start_positions.append(token_start_index)
                end_positions.append(token_end_index)

    tokenized_inputs["start_positions"] = start_positions
    tokenized_inputs["end_positions"] = end_positions
    return tokenized_inputs


tokenized = raw.map(preprocess_batch, batched=True, remove_columns=raw["train"].column_names)
tokenized

Map:   0%|          | 0/5 [00:00<?, ? examples/s]

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'offset_mapping', 'start_positions', 'end_positions'],
        num_rows: 5
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'offset_mapping', 'start_positions', 'end_positions'],
        num_rows: 1
    })
})

In [ ]:
#training part
import numpy as np
import evaluate
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer

model_ckpt = "deepset/roberta-base-squad2"
model = AutoModelForQuestionAnswering.from_pretrained(model_ckpt)

# Metrics (EM/F1) for QA
metric = evaluate.load("squad_v2")  # works for SQuAD v2; also fine for v1 if no unanswerables

def postprocess_predictions(examples, features, predictions, n_best_size=20, max_answer_length=50):
    """
    Quick post-processing for evaluation. For production, consider HF's official QA post-process utils.
    """
    from collections import defaultdict
    start_logits, end_logits = predictions
    example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
    features_per_example = defaultdict(list)
    for i, f_id in enumerate(features["overflow_to_sample_mapping"]):
        features_per_example[f_id].append(i)

    preds = {}
    for example_index, example_id in enumerate(examples["id"]):
        feature_indices = features_per_example.get(example_index, [])
        best_score = -1e9
        best_answer = ""
        context = examples["context"][example_index]

        for fi in feature_indices:
            starts = start_logits[fi]
            ends = end_logits[fi]
            start_indexes = np.argsort(starts)[-1:-n_best_size-1:-1]
            end_indexes = np.argsort(ends)[-1:-n_best_size-1:-1]
            for s in start_indexes:
                for e in end_indexes:
                    if e < s or (e - s + 1) > max_answer_length:
                        continue
                    score = starts[s] + ends[e]
                    # naive char span mapping with fast tokenizer
                    # for a production system, use offset_mapping from tokenizer
                    # Here we fallback: output just the top scoring span indexes
                    if score > best_score:
                        best_score = score
                        best_answer = ""  # set later as we didn't compute char offsets here
        # Fallback: empty; metrics won't be perfect without proper offsets (see note below)
        preds[example_id] = best_answer
    return preds

# NOTE:
# For proper EM/F1 evaluation you should use offset mappings from the tokenizer to extract exact spans.
# Keeping training simple here; you can still train and then evaluate qualitatively or add full post-processing later.

args = TrainingArguments(
    output_dir="./bert-system-design",
    eval_strategy="epoch",   # or "epoch"
    eval_steps=50,                  # run eval every 50 steps
    logging_strategy="steps",       # <– enable per-step logging
    logging_steps=10,               # <– print every 10 steps
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    fp16=True,
    load_best_model_at_end=True,
    report_to="none",               # avoid WandB popups
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized["train"],
    eval_dataset=tokenized["validation"],
    tokenizer=tokenizer
)

trainer.train()

/tmp/ipython-input-2561891385.py:70: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `RobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Epoch,Training Loss,Validation Loss
1,No log,0.711914
2,No log,0.711914
3,No log,0.711914


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `RobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `RobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


TrainOutput(global_step=3, training_loss=2.3478902180989585, metrics={'train_runtime': 27.907, 'train_samples_per_second': 0.538, 'train_steps_per_second': 0.108, 'total_flos': 3919451351040.0, 'train_loss': 2.3478902180989585, 'epoch': 3.0})

In [ ]:
#save and download
from google.colab import files
model.save_pretrained("bert-system-design")
tokenizer.save_pretrained("bert-system-design")
!zip -qr bert-system-design.zip bert-system-design
files.download("bert-system-design.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>